In [1]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

NUM_THRESHOLDS = 6

#reads in the data
data = pd.read_csv('movie_metadata.csv')
print(list(data))
print(data.isnull().sum().sum())
#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']
2698
Done


# Changing Genres into individual Columns

In [2]:
# make a set with all unique genres

genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

set(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adventure', 'News', 'Reality-TV', 'Thriller', 'Western', 'Mystery', 'Short', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Fantasy', 'Game-Show', 'Sport', 'Biography'])


In [3]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [4]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    tempcopy = data.copy()
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    removed_films = tempcopy.drop(tempcopy.index[remove_indices])
    
    print("Length after deletion: {}".format(len(data)))
    
    return data

tempcopy = movie_data.copy()
movie_data = []
for i in range(NUM_THRESHOLDS):
    tempremoved = remove_too_many_NaN(tempcopy.copy(), i + 1)
    movie_data.append(tempremoved)

Length before deletion: 5043
Length after deletion: 3771
Length before deletion: 5043
Length after deletion: 4490
Length before deletion: 5043
Length after deletion: 4777
Length before deletion: 5043
Length after deletion: 4900
Length before deletion: 5043
Length after deletion: 4983
Length before deletion: 5043
Length after deletion: 5023


# Plotting number of NaNs per threshold

In [5]:
def plot_num_NaNs(movie_data, marked):
    nans = []
    thresh = []
    samps = []
    for i in range(NUM_THRESHOLDS):
        numnans = movie_data[i].isnull().sum().sum()
        nans.append(numnans)
        samps.append(len(movie_data[i]))
        thresh.append(i + 1)
    
    fig = plt.figure()
#     ax = fig.add_subplot(121)
#     plt.plot(thresh, nans, 'bx-')
#     plt.plot(thresh[3], nans[3], 'ro')
#     for xy in zip(thresh, nans):
#         ax.annotate('%s' % xy[0], xy=xy, textcoords='data')
#     plt.ylabel('Number of NaNs')
#     plt.xlabel('Threshold')
    
    ax2 = fig.add_subplot(111)
    plt.plot(thresh, samps, 'rx-')
    plt.plot(thresh[marked], samps[marked], 'bo')
    for xy in zip(thresh, samps):
        ax2.annotate('%s' % xy[0], xy=xy, textcoords='data')
    plt.ylabel('Number of Samples')
    plt.xlabel('Threshold')
    plt.show()

    
plot_num_NaNs(movie_data, 3)

# Changing Color to Numeric

In [6]:
def numeric_color(movie_data):
    #replaces the color value to 1
    movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
    movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

    #replaces the NaN or black and white values to 0
    for item in movie_data.color:
        if item != 1:
            movie_data.color = movie_data.color.replace(to_replace = item, value = 0)

    #movie_data.color = new_color_column 
    #print(movie_data['color'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None
    print('Done')
    return movie_data

for i in range(NUM_THRESHOLDS):
    movie_data[i] = numeric_color(movie_data[i])

Done
Done
Done
Done
Done
Done


# Changing Country to Numeric

In [7]:
def numeric_country(movie_data):
    #replaces the USA values to 1
    movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
    movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
    #replaces the NaN or non USA values to 0
    for item in movie_data.country:
        if item != 1:
            movie_data.country = movie_data.country.replace(to_replace = item, value = 0)

    #print(movie_data['country'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None

    print('Done')
    return movie_data

for i in range(NUM_THRESHOLDS):
    movie_data[i] = numeric_country(movie_data[i])

Done
Done
Done
Done
Done
Done


# Changing Languange to Numeric

In [8]:
def numeric_language(movie_data):
    #replaces the English values to 1
    movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
    movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

    #replaces the other values to 0
    for item in movie_data.language:
        if item != 1:
            movie_data.language = movie_data.language.replace(to_replace = item, value = 0)

    #print(movie_data['language'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None
    print('Done')
    return movie_data

for i in range(NUM_THRESHOLDS):
    movie_data[i] = numeric_language(movie_data[i])

Done
Done
Done
Done
Done
Done


# Chaning Content_Rating to Numeric

In [9]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

for i in range(NUM_THRESHOLDS):
    movie_data[i] = content_to_numerical(movie_data[i])

# Replacing NaNs with averages

In [10]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes','gross', 'title_year'
                   , 'duration', 'facenumber_in_poster', 'content_rating']

for i in range(NUM_THRESHOLDS):
    for column in to_replace_NaNs:
        movie_data[i][column] = replace_NaNs(movie_data[i][column])

7722.74303898
2014.86157518
768.95332803
2.11111376293
46074665.6696
11486
804.246353752
52410371.7385
2003.01060727
110.219570406
1.37708830549
14
7032.59888641
1792.34922049
691.49108337
2.1096122633
41840819.2828
10423
744.864587973
49287524.5918
2001.97906459
109.24142539
1.35540691193
13.7810398924
6747.99685995
1713.49706868
664.698322851
2.11774347826
40444270.7169
10003
703.776847394
48619441.8755
2002.19866025
108.555276382
1.36479966436
13.6992675571
6648.62979592
1684.1759902
653.779595175
2.1284344634
39892924.8304
9849
692.662510221
48514997.6517
2002.39901881
108.243568804
1.36272504092
13.6882617062
6558.490263
1664.34712736
648.761167002
2.19000853607
39813189.6557
9728
688.57729027
48491666.3264
2002.45062983
107.495680129
1.37183098592
13.6780054933
6577.96372334
1656.08695652
646.135037954
2.2200721562
39769410.9927
9727
686.888168558
48468407.5268
2002.46838265
107.390618762
1.37005988024
13.6774261603


# Delete columns with too little 1's

In [11]:
def del_sparse_cols(movie_data, doprint=False):
    for genre in genres_set:
        if doprint:
            print(genre, ':')
        counter = 0
        for value in movie_data[genre]:
            if value == 1.0:
                counter+=1
        
        if doprint:
            print(counter)

    weghalen = ['Game-Show', 'News', 'Reality-TV', 'Short', 'Film-Noir']

    for name in weghalen:
        del movie_data[name]

    #print(movie_data)
    return movie_data

for i in range(NUM_THRESHOLDS):
    movie_data[i] = del_sparse_cols(movie_data[i])


In [12]:
def plot_cor(data):
    plt.figure(figsize=(10,10), tight_layout=True)
    plt.pcolor(data.corr())
    plt.yticks(np.arange(0.5, len(list(data)), 1), list(data))
    plt.xticks(np.arange(0.5, len(list(data)), 1), list(data), rotation=90)
    plt.show()

#plot_cor(movie_data_4)

# Trainset en testset

In [13]:
def split_to_sets(movie_data):
    movie_data_sets = {'movie_training': 0, 'ratings_training': 0, 
                       'movie_validation': 0, 'ratings_validation': 0, 
                       'movie_test': 0, 'ratings_test': 0}
    
    ratings = movie_data['imdb_score'].values
    del movie_data['imdb_score']

    X = movie_data.values
    X_std = StandardScaler().fit_transform(X)

    number_of_samples = len(ratings)
    np.random.seed(0)
    random_indices = np.random.permutation(number_of_samples)
    num_training_samples = round(number_of_samples*0.6)
    num_validation_samples = round(number_of_samples*0.2)+num_training_samples

    movie_data_sets['movie_training'] = X_std[random_indices[:num_training_samples]]
    ratings_training = ratings[random_indices[:num_training_samples]]

    movie_data_sets['movie_validation'] = X_std[random_indices[num_training_samples:num_validation_samples]]
    movie_data_sets['ratings_validation'] = ratings[random_indices[num_training_samples:num_validation_samples]]

    movie_data_sets['movie_test'] = X_std[random_indices[num_validation_samples:]]
    movie_data_sets['ratings_test'] = ratings[random_indices[num_validation_samples:]]

    movie_data_sets['ratings_training'] = list(ratings_training)

    print(len(movie_data_sets['ratings_training']))
    print(len(movie_data_sets['ratings_validation']))
    print(len(movie_data_sets['ratings_test']))
    
    return movie_data_sets

sets = []
for i in range(NUM_THRESHOLDS):
    sets.append(split_to_sets(movie_data[i]))

2263
754
754
2694
898
898
2866
955
956
2940
980
980
2990
997
996
3014
1005
1004


/home/carlo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/carlo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/carlo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/carlo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/carlo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/carlo/anaconda2/lib/python2.7/site-packages/

# Leren

In [14]:
def relative_error(y_predict, y):
    
    error = 0

    for i in range(len(y)):
        error += (abs(y_predict[i]-y[i]))/y[i]
    training_error = error/len(y)*100
    
    return training_error

In [15]:
def analyse_predictions(regr, sets, statdict, doprint=False):
    x_test = sets['movie_test']
    y_test = sets['ratings_test']
    
    regr.fit (sets['movie_training'], sets['ratings_training'])

    prediction = regr.predict(x_test)
    results_list.append(prediction)

    # The mean squared error
    print("Mean squared error: %.2f"
          % np.mean((prediction - y_test) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % regr.score(x_test, y_test))

    
    
    predres = list(regr.predict(x_test))
    actual = list(y_test)
    highest = avg = error = 0
    lowest = predres[0]
    for i in range(len(predres)):
        diff = abs(predres[i] - actual[i])
        avg += diff
        error += abs(predres[i] - actual[i]) / actual[i]
        
        if doprint:
            print("predicted:", predres[i]) 
            print ("\tActual:", actual[i])
            print ("\tDifference:", abs(predres[i] - actual[i]))

        if diff > highest:
            highest = diff
        if diff < lowest:
            lowest = diff

    errper = error / len(x_test) * 100
    avgdif =  avg / len(predres)
    
    if doprint:
        #print "Number of Samples:", len(predres)
        #print "Highest difference:", highest
        #print "Lowest difference:", lowest
        #print "average difference:", avgdif
        print("Error Percentage:", errper)
    
#     statdict['highest'].append(highest)
#     statdict['lowest'].append(lowest)
#     statdict['avgdif'].append(avgdif)
    statdict['errper'].append(errper)
    
    return statdict




results_list = []
statdicts = []
for i in range(NUM_THRESHOLDS):
    statdict = {'names':[], 'errper': [], 'predictions': []}
    statdicts.append(statdict)

### neural networks

In [16]:
# from sklearn.neural_network import MLPRegressor

# def neural(statdict, sets):

#     neural = MLPRegressor(hidden_layer_sizes =(100), activation = 'logistic', solver = 'adam', max_iter = 1000)

#     neural.fit(sets['movie_training'], sets['ratings_training'])
#     y_neural_train = neural.predict(sets['movie_training'])
#     y_neural_test = neural.predict(sets['movie_test'])

#     training_error = relative_error(y_neural_train, sets['ratings_training'])

#     print("Train error = "+'{}'.format(training_error)+" percent"+" in neural network algorithm")

#     test_error = relative_error(y_neural_test,sets['ratings_test'])

#     print("Test error = "'{}'.format(test_error)+" percent"+" in neural network algorithm")
    
#     statdict['errper'].append(test_error)
#     statdict['names'].append('neural')
#     statdict['predictions'].append(y_neural_test)
    
#     return statdict

# for i in range(4):
#     statdicts[i] = neural(statdicts[i], sets[i])

### k-nearest neighbors

In [17]:
def do_knn(statdict, sets, n_neighbors=5, weights='uniform'):
        
    neighbors = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
    neighbors.fit(sets['movie_training'], sets['ratings_training'])
    y_neighbors_train = neighbors.predict(sets['movie_training'])
    #y_predict_train = list(y_predict_train)

    train_error = relative_error(y_neighbors_train,sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in knn algorithm")

    y_neighbors_test = neighbors.predict(sets['movie_test'])
    #y_predict_test = list(y_predict_test)

    test_error = relative_error(y_neighbors_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

    statdict['errper'].append(test_error)
    statdict['names'].append('kNN')
    statdict['predictions'].append(y_neighbors_test)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_knn(statdicts[i], sets[i], n_neighbors=15, weights='distance')

Train error = 0.0 percent in knn algorithm
Test error = 11.0490640796 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 11.7695538866 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 13.3180628211 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 11.8789017784 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 13.4150840723 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 13.3765837677 percent in knn algorithm


### Support vector machine

In [18]:
from sklearn.svm import SVR, NuSVR, LinearSVR

def do_svr(statdict, sets):
    svr = SVR()
    svr.fit(sets['movie_training'], sets['ratings_training'])
    y_svr_train = svr.predict(sets['movie_training'])
    #y_train = list(y_train)

    train_error = relative_error(y_svr_train, sets['ratings_training'])

    print ("Train error = "+'{}'.format(train_error))

    y_svr_test = svr.predict(sets['movie_test'])
    #y_test = list(y_test)

    test_error = relative_error(y_svr_test, sets['ratings_test'])

    print ("Test error = "+'{}'.format(test_error))

    statdict['errper'].append(test_error)
    statdict['names'].append('SVM')
    statdict['predictions'].append(y_svr_test)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_svr(statdicts[i], sets[i])

Train error = 8.18761939314
Test error = 10.2504894917
Train error = 8.61554189663
Test error = 10.9147868316
Train error = 8.69726527135
Test error = 12.1802303446
Train error = 9.15012885363
Test error = 10.8586127884
Train error = 9.089494534
Test error = 12.6239888385
Train error = 9.05028697238
Test error = 12.6123027094


### Decision trees

In [19]:
from sklearn.tree import DecisionTreeRegressor

def do_dt(statdict, sets):
    trees = DecisionTreeRegressor()
    trees.fit(sets['movie_training'], sets['ratings_training'])
    y_trees_train = trees.predict(sets['movie_training'])

    train_error = relative_error(y_trees_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in decision trees")

    y_trees_test = trees.predict(sets['movie_test'])

    test_error = relative_error(y_trees_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in decision trees")

    statdict['errper'].append(test_error)
    statdict['names'].append('DT')
    statdict['predictions'].append(y_trees_test)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_dt(statdicts[i], sets[i])

Train error = 4.53099272772e-16 percent in decision trees
Test error = 12.938106782 percent in decision trees
Train error = 3.99570033265e-16 percent in decision trees
Test error = 13.0720374377 percent in decision trees
Train error = 3.85960584977e-16 percent in decision trees
Test error = 15.1725795103 percent in decision trees
Train error = 3.4713944456e-16 percent in decision trees
Test error = 14.6848133608 percent in decision trees
Train error = 3.37434601533e-16 percent in decision trees
Test error = 15.9992074466 percent in decision trees
Train error = 3.48625318472e-16 percent in decision trees
Test error = 15.2397145098 percent in decision trees


### Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor

def do_randforest(statdict, sets):
    random_forest = RandomForestRegressor(n_estimators = 20)
    random_forest.fit(sets['movie_training'], sets['ratings_training'])
    y_forest_train = random_forest.predict(sets['movie_training'])

    train_error = relative_error(y_forest_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_forest_test = random_forest.predict(sets['movie_test'])

    test_error = relative_error(y_forest_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('Rfor')
    statdict['predictions'].append(y_forest_test)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_randforest(statdicts[i], sets[i])

Train error = 3.73059015199 percent in random forest
Test error = 9.81629476237 percent in random forest
Train error = 3.85245507305 percent in random forest
Test error = 10.4878573693 percent in random forest
Train error = 3.88312977613 percent in random forest
Test error = 11.4802460286 percent in random forest
Train error = 4.01885790857 percent in random forest
Test error = 10.5670314125 percent in random forest
Train error = 3.94779742525 percent in random forest
Test error = 11.9421101078 percent in random forest
Train error = 3.97121737625 percent in random forest
Test error = 11.7071565053 percent in random forest


### Extremely randomized trees

In [21]:
def do_xrandtrees(statdict, sets):
    extra = ExtraTreesRegressor(n_estimators = 20)
    extra.fit(sets['movie_training'], sets['ratings_training'])
    y_extra_train = extra.predict(sets['movie_training'])

    train_error = relative_error(y_extra_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_extra_test = extra.predict(sets['movie_test'])

    test_error = relative_error(y_extra_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('XRfor')
    statdict['predictions'].append(y_extra_test)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_xrandtrees(statdicts[i], sets[i])

Train error = 1.78592811955e-14 percent in random forest
Test error = 9.89456521061 percent in random forest
Train error = 1.80124713131e-14 percent in random forest
Test error = 10.1359639964 percent in random forest
Train error = 1.79403908702e-14 percent in random forest
Test error = 11.6106181548 percent in random forest
Train error = 1.82594909424e-14 percent in random forest
Test error = 10.1771437884 percent in random forest
Train error = 1.79587063332e-14 percent in random forest
Test error = 11.8318383864 percent in random forest
Train error = 1.77806739716e-14 percent in random forest
Test error = 11.868334368 percent in random forest


### Bagging regressor

In [22]:
def do_baggingregr(statdict, sets):
    #regressor  = BaggingRegressor(MLPRegressor(max_iter = 300))
    bagging = BaggingRegressor(RandomForestRegressor())
    bagging.fit(sets['movie_training'], sets['ratings_training'])
    y_bagging_train = bagging.predict(sets['movie_training'])
    #y_predict_train = list(y_predict_train)

    train_error = relative_error(y_bagging_train ,sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_bagging_test = bagging.predict(sets['movie_test'])
    #y_predict_test = list(y_predict_test)

    test_error = relative_error(y_bagging_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('bag')
    statdict['predictions'].append(y_bagging_test)
    
    return statdict

# for i in range(4):
#     statdicts[i] = do_baggingregr(statdicts[i], sets[i])

# Validation

# Ensemble

### Neural networks and svr and random forest

In [23]:
def do_ensemble(statdict, sets):
    y_svr_test = statdict['predictions'][statdict['names'].index('SVM')]
    y_forest_test = statdict['predictions'][statdict['names'].index('Rfor')]
    y_extra_test = statdict['predictions'][statdict['names'].index('XRfor')]
    ratings_test = sets['ratings_test']


    prediction = [sum(x)/3 for x in zip(y_svr_test, y_forest_test, y_extra_test)]

    counter = 0

    for i in range(len(prediction)):
        if abs(prediction[i]-ratings_test[i])>1.0:
            counter+= 1
            #print(prediction[i], ratings_test[i])

    error_ensemble = relative_error(prediction, ratings_test)

    statdict['errper'].append(error_ensemble)
    statdict['names'].append('ensemble')
    statdict['predictions'].append(prediction)

    print(error_ensemble)
    print(counter/980.0)
    
    return statdict

for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_ensemble(statdicts[i], sets[i])

9.31194243467
0.0938775510204
9.93221961946
0.113265306122
11.2447081488
0.151020408163
10.031814202
0.161224489796
11.5486219572
0.167346938776
11.4932168494
0.172448979592


In [24]:
# print(movie_data.columns)

In [25]:
# movie_data['color'].describe()

In [26]:
from sklearn import linear_model

def do_linear_models(statdict, sets):
    regr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('linreg')


    regr = linear_model.Ridge()
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('ridgereg')


    regr = linear_model.Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000000,
       normalize=False, positive=False, precompute=False, random_state=None,
       selection='cyclic', tol=0.0001, warm_start=False)
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('lassoreg')


    regr = linear_model.BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
           fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
           normalize=False, tol=0.001, verbose=False)
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('bayesian')
    
    return statdict


for i in range(NUM_THRESHOLDS):
    statdicts[i] = do_linear_models(statdicts[i], sets[i])

Mean squared error: 0.69
Variance score: 0.37
Mean squared error: 0.69
Variance score: 0.38
Mean squared error: 0.68
Variance score: 0.39
Mean squared error: 0.67
Variance score: 0.39
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.87
Variance score: 0.34
Mean squared error: 0.87
Variance score: 0.33
Mean squared error: 0.88
Variance score: 0.33
Mean squared error: 0.88
Variance score: 0.33
Mean squared error: 0.77
Variance score: 0.38
Mean squared error: 0.77
Variance score: 0.38
Mean squared error: 0.78
Variance score: 0.38
Mean squared error: 0.78
Variance score: 0.38
Mean squared error: 0.96
Variance score: 0.31
Mean squared error: 0.96
Variance score: 0.31
Mean squared error: 0.96
Variance score: 0.31
Mean squared error: 0.96
Variance score: 0.31
Mean squared error: 0.94
Variance score: 0.31
Mean squared error: 0.94
Variance 

In [32]:
def plot_threshold(statdicts):
    n_groups = len(statdicts[0]['names'])

    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    
    index = np.arange(n_groups)
    bar_width = 0.9 * (1.0 / len(statdicts))

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    colors = ['b', 'g', 'r', 'c', 'm', 'k', 'y', 'w', 'b', 'g', 'r', 'c', 'm', 'k', 'y', 'w']
    for i in range(len(statdicts)):
        rects1 = plt.bar(index + bar_width * i, statdicts[i]['errper'], bar_width,
                         alpha=opacity,
                         color=colors[i],
                         error_kw=error_config,
                         label="threshold {}".format(i + 1))
    plt.xlabel('Algorithm')
    plt.ylabel('Error Percentages (in %)')
    plt.title('Error Percentages for various algorithms depending on the threshold')
    plt.xticks(index + bar_width * 2, statdict['names'], rotation=45)
    plt.tight_layout()
    plt.legend(bbox_to_anchor=(0, 0.5), loc=2, borderaxespad=0.)
    plt.show()
    
plot_threshold(statdicts)

In [30]:
def plot_errors(statdict):
    n_groups = len(statdict['names'])

    fig, ax = plt.subplots()
    
    index = np.arange(n_groups)
    bar_width = 0.9

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, statdict['errper'], bar_width,
                     alpha=opacity,
                     color='r',
                     error_kw=error_config)

    plt.xlabel('Algorithm')
    plt.ylabel('Error Percentages (in %)')
    plt.title('Error Percentages for various algorithms using threshold 4')
    plt.xticks(index + bar_width * 0.5, statdict['names'], rotation=90)
    plt.tight_layout()
    plt.show()

plot_errors(statdicts[3])

In [29]:
def plot_algorithm(statdicts, algorithm):
    n_groups = NUM_THRESHOLDS

    errpers = []
    algind = statdicts[0]['names'].index(algorithm)
    for t in range(n_groups):
        errpers.append(statdicts[t]['errper'][algind])
        
    fig, ax = plt.subplots()
    
    index = np.arange(n_groups)
    bar_width = 1

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, errpers, bar_width,
                     alpha=opacity,
                     color='r',
                     error_kw=error_config)

    plt.xlabel('Threshold')
    plt.ylabel('Error Percentages (in %)')
    plt.title('Error Percentages for algorithm \'{}\' with varying thresholds'.format(algorithm))
    plt.xticks(index + bar_width * 0.5, range(1, n_groups + 1))
    plt.tight_layout()
    plt.show()

plot_algorithm(statdicts, 'ensemble')